In [1]:
import datasets
import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM

import mlm
from mlm.scorers import MLMScorerPT 
from mlm.models import get_pretrained

import mxnet as mx
ctxs = [mx.cpu()]

/Users/guydavidson/opt/anaconda3/envs/torch/lib/python3.8/site-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


## NOTES

* **TODO**: verify the surgery I did to the mlm.scorers codebase to accept RoBERTa is legit
* If we want other models, we'll have to add them there, too, perhaps with a bit more work if their output format is very different
* The function below implements the very basic test. Next steps I can see us wanting to do:
    * Combine it with sentences from the real data
    * Check at least two alternative word-replacement strategies (within category, between categories)
    * Write more of a pipeline that samples words, sentences, replacement words for each sentence, and spits out scorers
* Open questions:
    * How do we measure how well the model did? Rank of the correct sentence? NLL difference from correct and other best-performing sentence? Both? 

In [3]:


checkpotint_name = "nyu-mll/roberta-base-100M-1"
# checkpotint_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpotint_name)
model = AutoModelForMaskedLM.from_pretrained(checkpotint_name)

In [4]:
scorer = MLMScorerPT(model, None, tokenizer, ctxs)

In [5]:
scorer.score_sentences(['Here is a test sentence'])

[-18.295551404356956]

In [6]:
scorer.score_sentences(['Here is a purple potato'])

[-25.806969970464706]

In [9]:
def score_sentence_alternatives(scorer, sentence, target_word, alt_words):
    if not target_word in sentence:
        raise ValueError(f'The target word "{target_word}" is not in the sentence "{sentence}"')
    
    sentence_copies = [sentence.replace(target_word, alt) for alt in alt_words]
    scores = scorer.score_sentences([sentence] + sentence_copies)
    return scores


In [10]:
score_sentence_alternatives(scorer, 'The toy is on the table', 'toy', ['cow', 'dad', 'banana'])

[-17.26363129913807,
 -15.599759384989738,
 -24.108910769224167,
 -16.903809688985348]